The problem of graph clustering for social network graphs like `page-page` naturally moulds into community detection. There are a number of community detection algorithms. We'll test the [Girvan Newman's Algorithm](https://en.wikipedia.org/wiki/Girvan%E2%80%93Newman_algorithm).

Note: This notebook continues from `visual.ipynb`

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
target = pd.read_csv('../facebook_large/musae_facebook_target.csv')
edges  = pd.read_csv('../facebook_large/musae_facebook_edges.csv')

In [4]:
G = nx.Graph()
for it, cat in zip(target['id'], target['page_type']):
    G.add_node(it, page_type=cat)
for n1, n2 in zip(edges['id_1'], edges['id_2']):
    G.add_edge(n1, n2)

In [7]:
# sanity check
G.number_of_nodes(), G.number_of_edges()

(22470, 171002)